In [1]:
import pyspark
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
NESSIE_URI = "http://nessie:19120/api/v1"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"



In [2]:
conf = (
    pyspark.SparkConf()
        .setAppName('testapp')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
        .set('spark.sql.catalog.nessie.s3.endpoint', 'http://minio:9000')
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
  		#MINIO CREDENTIALS
        .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/home/docker/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/docker/.ivy2/cache
The jars for the packages stored in: /home/docker/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f41011cc-cf45-4a5d-8853-69ea11173c49;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.3.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12;0.67.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central

25/04/25 17:53:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark Running


In [3]:
## LOAD A CSV INTO AN SQL VIEW
csv_df = spark.read.format("csv").option("header", "true").load("../datasets/df_open_2023.csv")
csv_df.createOrReplaceTempView("csv_open_2023")


In [4]:

## CREATE AN ICEBERG TABLE FROM THE SQL VIEW
spark.sql("CREATE TABLE IF NOT EXISTS nessie.df_open_2023_lesson2 USING iceberg AS SELECT * FROM csv_open_2023").show()


SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".    (0 + 4) / 4]
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


++
||
++
++



In [5]:

## QUERY THE ICEBERG TABLE
spark.sql("SELECT * FROM nessie.df_open_2023_lesson2 limit 10").show()


+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|competitorId|   competitorName|firstName|   lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|       affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+-----------------+---------+-----------+------+------+-------------------+-------------------+--------+------------------+-----------+--------------------+---+------+------+-----------+------------+--------+----+
|      469656|    Jeffrey Adler|  Jeffrey|      Adler|   ACT|     M|                 CA|             Canada|      35|North America East|      18059| CrossFit Wonderland| 29| 69 in|197 lb|          1|         107|       1|2023|
|      300638|   Tola Morakinyo|     Tola|  Morakinyo|   ACT|     M|                 US|    

In [6]:
def e(qry):
    spark.sql(qry).show()

In [ ]:

# ## QUERY THE COUNT OF ENTRIES
# spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()

# ## CREATE A BRANCH WITH NESSIE
# spark.sql("CREATE BRANCH IF NOT EXISTS lesson2 IN nessie")

# ## SWTICH TO THE NEW BRANCH
# spark.sql("USE REFERENCE lesson2 IN nessie")

# ## DELETE ALL RECORDS WHERE countryOfOriginCode = 'FR'
# spark.sql("DELETE FROM nessie.df_open_2023_lesson2 WHERE countryOfOriginCode = 'FR'")

# ## QUERY THE COUNT OF ENTRIES
# spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()

# ## SWITCH BACK TO MAIN BRANCH
# spark.sql("USE REFERENCE main IN nessie")

# ## QUERY THE COUNT OF ENTRIES
# spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()

# ## MERGE THE CHANGES
# spark.sql("MERGE BRANCH lesson2 INTO main IN nessie")

# ## QUERY THE COUNT OF ENTRIES
# spark.sql("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2").show()

In [7]:
e("select count(1) as total from nessie.df_open_2023_lesson2")

+------+
| total|
+------+
|302231|
+------+



In [8]:
e("create branch if not exists lesson2 in nessie")

+-------+-------+--------------------+
|refType|   name|                hash|
+-------+-------+--------------------+
| Branch|lesson2|51f6bc3e1d5a97dad...|
+-------+-------+--------------------+



In [9]:
e("use reference lesson2 in nessie")

+-------+-------+--------------------+
|refType|   name|                hash|
+-------+-------+--------------------+
| Branch|lesson2|51f6bc3e1d5a97dad...|
+-------+-------+--------------------+



In [13]:
e("select * from nessie.df_open_2023_lesson2 limit 1 ")

+------------+--------------+---------+--------+------+------+-------------------+-------------------+--------+------------------+-----------+-------------------+---+------+------+-----------+------------+--------+----+
|competitorId|competitorName|firstName|lastName|status|gender|countryOfOriginCode|countryOfOriginName|regionId|        regionName|affiliateId|      affiliateName|age|height|weight|overallRank|overallScore|genderId|year|
+------------+--------------+---------+--------+------+------+-------------------+-------------------+--------+------------------+-----------+-------------------+---+------+------+-----------+------------+--------+----+
|      469656| Jeffrey Adler|  Jeffrey|   Adler|   ACT|     M|                 CA|             Canada|      35|North America East|      18059|CrossFit Wonderland| 29| 69 in|197 lb|          1|         107|       1|2023|
+------------+--------------+---------+--------+------+------+-------------------+-------------------+--------+---------

In [14]:
spark.sql("DELETE FROM nessie.df_open_2023_lesson2 WHERE countryOfOriginCode = 'FR'")

DataFrame[]

In [15]:
e("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2")

+------+
| Total|
+------+
|280003|
+------+



In [16]:
e("USE REFERENCE main IN nessie")

+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch|main|51f6bc3e1d5a97dad...|
+-------+----+--------------------+



In [17]:
e("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2")

+------+
| Total|
+------+
|302231|
+------+



In [18]:
e("MERGE BRANCH lesson2 INTO main IN nessie")

+----+--------------------+
|name|                hash|
+----+--------------------+
|main|f07157311a671b80f...|
+----+--------------------+



In [19]:
e("SELECT Count(*) as Total FROM nessie.df_open_2023_lesson2")

+------+
| Total|
+------+
|280003|
+------+

